In [14]:
import datetime

fecha_actual = datetime.date.today()
un_dia       = datetime.timedelta(days=1)

ANIO_INICIO = fecha_actual.year - 4
MES_INICIO  = fecha_actual.month
DIA_INICIO  = fecha_actual.day 

fecha_inicio = datetime.date(ANIO_INICIO, MES_INICIO, DIA_INICIO)

In [16]:
while fecha_inicio <= fecha_actual:
    print(fecha_inicio)
    fecha_inicio += un_dia
